# Notebook de Características

En este notebook extraigo mediante web scraping datos y características del listado de vehículos obtenido en la limpieza del dataframe EDA_Extraccion_A

In [1]:
# Importo las librerias
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

In [2]:
# La idea es generar un DF más completo con los vehículos que he sacado
# Encontré una web muy completa
# Utilizo el listado de vehículos del df_EDA

In [3]:
# Creo diferentes funciones para automatizar el proceso
# Funciones adicionales
# Función para comprobar la accesibilidad
def accesibilidad (url):
    conexion = requests.get(url)
    print(conexion.status_code, conexion.reason)

# Función para extraer datos de características
def extraccion (url):
    conexion = requests.get(url)
    sopa = bs(conexion.text, "lxml")
    datos = sopa.find_all( 'span', class_="text-11") 
    lista_datos = []
    for indice, dato in enumerate(datos):
        lista_datos.append(dato.text.strip())
        indice += 1
    return lista_datos

# Función para extraer características 2
def extraccion_2 (url):
    conexion = requests.get(url)
    sopa = bs(conexion.text, "lxml")
    datos = sopa.find_all( 'dd', class_="font-bold whitespace-nowrap lg:text-17 xl:text-18") 
    lista_datos = []
    for indice, dato in enumerate(datos):
        lista_datos.append(dato.text.strip())
        indice += 1
    return lista_datos

# Función para extraer las etiquetas
def lista_etiquetas (url):
    conexion = requests.get(url)
    sopa = bs (conexion.text, 'lxml')
    etiquetado = sopa.find_all("img", {"alt": True})
# Creo la lista de las etiquetas
    nombre_pegatinas = []
    for nombre in etiquetado:
        pegatina = nombre["alt"] # recojo los nombres de las pegatinas con la primera letra de cada palabra en mayúscula ( para normaizar ya datos)
        nombre_pegatinas.append(pegatina)
        etiqueta = [x for x in nombre_pegatinas if "Etiqueta medioambiental " in x] # de la lista me quedo con las que se llaman Etiqueta medioambiental
        lista_etiquetas = list(set(etiqueta)) # me quedo solo con los valores únicos
    return lista_etiquetas

# Función para extraer el tipo de motor 
def lista_motores (url):
    conexion = requests.get(url)
    sopa = bs (conexion.text, 'lxml')
    motores = sopa.find_all('td', None)
# Creo la lista de los motores
    lista_motores = []
    for nombre in motores:
        lista_motores.append(nombre.text)
        motor_convencional = [x for x in lista_motores if "MHEV" in x] # de la lista me quedo con las que se llaman HEV
        motor_diesel = [x for x in lista_motores if "Diesel" in x] 
        motor_gasolina = [x for x in lista_motores if "Gasolina" in x] 
        motor_glp = [x for x in lista_motores if "LPG" in x] 
        motor_combinado = [x for x in lista_motores if "GLP" in x] 
        motor_bifuel = [x for x in lista_motores if "Bifuel" in x]
        motor_hibrido = [x for x in lista_motores if "PHEV" in x] # me quedo con los que tienen PHEV
        motor_electrico = [x for x in lista_motores if "BEV" in x] # me quedo con los que tiene BEV
        lista_completa = motor_convencional + motor_hibrido + motor_electrico + motor_diesel + motor_gasolina + motor_combinado + motor_glp + motor_bifuel
        listado_motores = list(set(lista_completa)) # me quedo solo con los valores únicos
    return listado_motores

# Función para extraer el tipo de conbustible (gasolina, diesel, glp, bifuel, eéctrico e híbrido)
def lista_combustibles (url):
    conexion = requests.get(url)
    sopa = bs (conexion.text, 'lxml')
    combustibles = sopa.find_all('td', None)
# Creo la lista de los motores
    lista_combustibles = []
    for nombre in combustibles:
        lista_combustibles.append(nombre.text)
        combustible_diesel = [x for x in lista_combustibles if "Diesel" in x] 
        combustible_gasolina = [x for x in lista_combustibles if "Gasolina" in x] 
        combustible_combinado = [x for x in lista_combustibles if "GLP" in x] 
        combustible_bifuel = [x for x in lista_combustibles if "Bifuel" in x]
        combustible_hibrido = [x for x in lista_combustibles if "Híbrido" in x] # me quedo con los que tienen PHEV
        combustible_electrico = [x for x in lista_combustibles if "Eléctrico" in x] # me quedo con los que tiene BEV
        lista_completa = combustible_diesel + combustible_gasolina + combustible_combinado + combustible_bifuel + combustible_hibrido + combustible_electrico
        listado_combustible = list(set(lista_completa)) # me quedo solo con los valores únicos
    return listado_combustible

# Función para extraer lo peor
def lo_peor (url):
    conexion = requests.get(url)
    sopa = bs (conexion.text, 'lxml')
    peores = sopa.find_all('div', class_="text-gray-dark text-16 sm:text-17 leading-5", limit=2)
    if peores != []: # por si no hay dato
        lista_mejorable = []
        for indice, peor in enumerate(peores):
            lista_mejorable.append(peor.text.strip())
            indice += 1
            mejorable = [lista_mejorable[i] for i in range(1, len(lista_mejorable), 2)] # me quedo con la segunda que corresponde con lo mejorable
    else:
        mejorable = None
    return mejorable

In [4]:
# Función principal para la extraccion de todos los datos
def datos (url, isofix):
    conexion = requests.get(url)
    sopa = bs(conexion.text, "lxml")
    marcas = []
    marca = sopa.find_all('a', class_="text-gray-dark font-bold")
    marcas.append(marca[1].text.strip())
    modelos =[]
    modelo = sopa.find('h1', class_="order-2 lg:order-1 leading-none lg:leading-[0.85] text-gray-800 flex-1 font-bold text-18 lg:text-48")
    modelos.append(((modelo.text.strip()).split())[2:]) # cogo la tercera palabra para quedare con el modelo
    lista_datos = extraccion(url)
    carrocerias = []
    carroceria = lista_datos[0]
    carrocerias.append(carroceria)
    largos = []
    largo = lista_datos[1]
    largos.append(largo)
    altos = []
    alto = lista_datos[2]
    altos.append(alto)
    otros_datos = extraccion_2(url)
    anchos = []
    ancho = otros_datos[2]
    anchos.append(ancho)
    pesos = []
    peso = otros_datos[3]
    pesos.append(peso)
    maleteros = []
    maletero = otros_datos[5]
    maleteros.append(maletero)
    depositos = []
    deposito = otros_datos[4]
    depositos.append(deposito)
    potencias = []
    potencia = lista_datos[4]
    potencias.append(potencia)
    plazas = []
    plaza = lista_datos[5]
    plazas.append(plaza)
    isofixes =[]
    isofixes.append(isofix)
    consumos = []
    consumo = lista_datos[6]
    consumos.append(consumo)
    combustibles = []
    combustible = lista_combustibles(url) 
    combustibles.append(combustible)
    motores = []
    motor  = lista_motores(url)
    motores.append(motor)
    emisiones = []
    emision = lista_datos[7]
    emisiones.append(emision)
    etiquetas = []
    etiqueta = lista_etiquetas(url)
    etiquetas.append(etiqueta)
    precios = []
    precio = sopa.find('a', class_="text-primary text-22 font-bold leading-none")
    precios.append(precio.text.strip())
    valoraciones = []
    valoracion = sopa.find('div', class_="shrink text-center text-42 xl:text-48 font-bold")
    valoraciones.append(valoracion.text.strip())
    lo_mejor = []
    mejor = sopa.find('div', class_="text-gray-dark text-16 sm:text-17 leading-5") # al poner sólo el find, me da el primer valor, que corresponde con lo mejor
    if mejor != None: # por si no hay
        lo_mejor.append(mejor.text.strip())
    else:
        lo_mejor.append(None)
    lo_mejorable = []
    peor = lo_peor(url)
    lo_mejorable.append(peor)
    opiniones = []
    opinion = sopa.find('p', class_="font-bold pr-4 pt-4")
    if opinion != None: # por si no hay
        opiniones.append(opinion.text.strip())
    else:
        opiniones.append(None)
    return marcas, modelos, carrocerias, largos, altos, anchos, pesos, maleteros, depositos, potencias, plazas, isofixes, consumos, motores, emisiones, etiquetas, precios, valoraciones, lo_mejor, lo_mejorable, opiniones, combustibles


In [5]:
# Creo las listas para el diccionario
lista_marcas = []
lista_modelos = []
lista_carrocerias = []
lista_largos = []
lista_altos = []
lista_anchos = []
lista_pesos = []
lista_maleteros = []
lista_depositos = []
lista_potencias = []
lista_plazas = []
lista_isofixes = []
lista_consumos = []
listado_combustible = []
listado_motores = []
lista_emisiones = []
listado_etiquetas = []
lista_precios = []
lista_valoraciones = []
lista_lo_mejor = []
lista_lo_mejorable = []
lista_opiniones = []


In [6]:
# Genero un diccionario con los datos obtenidos
diccionario_coches = {
    'Marca': lista_marcas,
    'Modelo': lista_modelos,
    'Carroceria': lista_carrocerias,
    'Largo': lista_largos,
    'Alto': lista_altos, 
    'Ancho': lista_anchos,
    'Peso': lista_pesos,
    'Maletero': lista_maleteros,
    'Depósito' : lista_depositos,
    'Potencia': lista_potencias,
    'Plazas': lista_plazas,
    'Isofix': lista_isofixes,
    'Consumo': lista_consumos,
    'Combustible': listado_combustible,
    'Motor': listado_motores,
    'Emisiones': lista_emisiones,
    'Etiqueta': listado_etiquetas,
    'Precio': lista_precios,
    'Valoración': lista_valoraciones,
    'Lo mejor': lista_lo_mejor,
    'Lo mejorable': lista_lo_mejorable,
    'Opinión': lista_opiniones,

}


In [7]:
# Función para introducir los datos en el diccionario
def diccionario (lista):
    lista_marcas.extend(lista[0])
    lista_modelos.extend(lista[1])
    lista_carrocerias.extend(lista[2])
    lista_largos.extend(lista[3])
    lista_altos.extend(lista[4])
    lista_anchos.extend(lista[5])
    lista_pesos.extend(lista[6])
    lista_maleteros.extend(lista[7])
    lista_depositos.extend(lista[8])
    lista_potencias.extend(lista[9])
    lista_plazas.extend(lista[10])
    lista_isofixes.extend(lista[11])
    lista_consumos.extend(lista[12])
    listado_combustible.extend(lista[21])
    listado_motores.extend(lista[13])
    lista_emisiones.extend(lista[14])
    listado_etiquetas.extend(lista[15])
    lista_precios.extend(lista[16])
    lista_valoraciones.extend(lista[17])
    lista_lo_mejor.extend(lista[18])
    lista_lo_mejorable.extend(lista[19])
    lista_opiniones.extend(lista[20])
    return lista_marcas, lista_modelos, lista_carrocerias,lista_largos, lista_altos,lista_anchos,lista_pesos,lista_maleteros,lista_depositos,lista_potencias,lista_plazas,lista_isofixes,lista_consumos,listado_motores, lista_emisiones,listado_etiquetas,lista_precios,lista_valoraciones,lista_lo_mejor,lista_lo_mejorable,lista_opiniones


In [8]:
# Uno todas las funciones en un único paso
def genera_datos (url,isofix):
    accesibilidad(url)
    data = datos(url, isofix) # los datos de Isofix los he buscado manualmente
    diccionario(data)
    

In [9]:
url_1 = 'https://www.quecochemecompro.com/precios/audi-q7/'
genera_datos(url_1, 3)
url_2 = 'https://www.quecochemecompro.com/precios/bmw-x5/'
genera_datos(url_2, 3)
url_3 = 'https://www.quecochemecompro.com/precios/bmw-x5-hibrido/'
genera_datos(url_3, 3)
url_4 = 'https://www.quecochemecompro.com/precios/bmw-x7/'
genera_datos(url_4, 4)
url_5 = 'https://www.quecochemecompro.com/precios/citroen-c3-aircross/'
genera_datos(url_5, 2)
url_6 = 'https://www.quecochemecompro.com/precios/citroen-c3-aircross-electrico/'
genera_datos(url_5, 2)
url_7 = 'https://www.quecochemecompro.com/precios/citroen-grand-c4-picasso/'
genera_datos(url_7, 2)
url_8 = 'https://www.quecochemecompro.com/precios/citroen-holidays/'
genera_datos(url_8, isofix=None)
url_9 = 'https://www.quecochemecompro.com/precios/citroen-spacetourer/'
genera_datos(url_9, 6)
url_10 = 'https://www.quecochemecompro.com/precios/citroen-spacetourer/'
genera_datos(url_10, 6)

200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK


In [10]:
url_11 = 'https://www.quecochemecompro.com/precios/citroen-e-berlingo/'
genera_datos(url_11,3)
url_12 = 'https://www.quecochemecompro.com/precios/dacia-jogger/'
genera_datos(url_12,2)
url_13 = 'https://www.quecochemecompro.com/precios/dacia-jogger-hibrido/'
genera_datos(url_13,2)
url_14 = 'https://www.quecochemecompro.com/precios/dacia-lodgy/'
genera_datos(url_14, 4)
url_15 = 'https://www.quecochemecompro.com/precios/dfsk-600/'
genera_datos(url_15, 2)
url_16 = 'https://www.quecochemecompro.com/precios/dfsk-e5/'
genera_datos(url_16, 4)
url_17 = 'https://www.quecochemecompro.com/precios/dfsk-glory-580/'
genera_datos(url_17, 4)
url_18 = 'https://www.quecochemecompro.com/precios/ebro-s800/'
genera_datos(url_18, 2)
url_19 = 'https://www.quecochemecompro.com/precios/ebro-s800-hibrido/'
genera_datos(url_19, 2)

200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK


In [11]:
url_20 ='https://www.quecochemecompro.com/precios/evo-7/'
genera_datos(url_20, 2)
url_21 = 'https://www.quecochemecompro.com/precios/fiat-ulysse/'
genera_datos(url_21, 4)
url_22 = 'https://www.quecochemecompro.com/precios/fiat-ulysse-electrico/'
genera_datos(url_22, 4)
url_23 = 'https://www.quecochemecompro.com/precios/ford-s-max/'
genera_datos(url_23, 3)
url_24 = 'https://www.quecochemecompro.com/precios/ford-tourneo-connect/'
genera_datos(url_24, 2)
url_25 = 'https://www.quecochemecompro.com/precios/ford-tourneo-custom/'
genera_datos(url_25, 5)
url_26 = 'https://www.quecochemecompro.com/precios/ford-tourneo-custom-electrico/'
genera_datos(url_26, 5)
url_27 = 'https://www.quecochemecompro.com/precios/ford-transit-connect/'
genera_datos(url_27, 4)
url_28 = 'https://www.quecochemecompro.com/precios/hongqi-ehs9/'
genera_datos(url_28, 2)
url_29 = 'https://www.quecochemecompro.com/precios/hyundai-kona/'
genera_datos(url_29, 2)
url_30 = 'https://www.quecochemecompro.com/precios/hyundai-kona-hibrido/'
genera_datos(url_30, 2)
url_31 = 'https://www.quecochemecompro.com/precios/hyundai-kona-electrico/'
genera_datos(url_31, 2)

200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK


In [12]:
url_32 = 'https://www.quecochemecompro.com/precios/hyundai-santa-fe-hibrido/'
genera_datos(url_32, 4)
url_33 = 'https://www.quecochemecompro.com/precios/hyundai-staria/'
genera_datos(url_33, 2)
url_34 = 'https://www.quecochemecompro.com/precios/ssangyong-rexton/'
genera_datos(url_34, 2)
url_35 = 'https://www.quecochemecompro.com/precios/kia-e-niro-electrico/'
genera_datos(url_35, 2)
url_36 = 'https://www.quecochemecompro.com/precios/kia-ev9/'
genera_datos(url_36, 4)
url_37 = 'https://www.quecochemecompro.com/precios/kia-sorento/'
genera_datos(url_36, 3)
url_37 = 'https://www.quecochemecompro.com/precios/kia-sorento-hibrido/'
genera_datos(url_37, 3)
url_38 = 'https://www.quecochemecompro.com/precios/land-rover-discovery-sport/'
genera_datos(url_38, 2)
url_39 = 'https://www.quecochemecompro.com/precios/land-rover-discovery-sport-hibrido/'
genera_datos(url_39, 2)

200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK


In [13]:
url_40 = 'https://www.quecochemecompro.com/precios/lexus-lm/'
genera_datos(url_40, 2)
url_41 = 'https://www.quecochemecompro.com/precios/mazda-cx-80/'
genera_datos(url_41, 2)
url_42 = 'https://www.quecochemecompro.com/precios/mazda-cx-80-hibrido/'
genera_datos(url_42, 2)
url_43 = 'https://www.quecochemecompro.com/precios/mercedes-glb/'
genera_datos(url_43, 4)
url_44 = 'https://www.quecochemecompro.com/precios/mercedes-clase-t/'
genera_datos(url_44, 3)
url_45 = 'https://www.quecochemecompro.com/precios/mercedes-clase-v/'
genera_datos(url_45, 4)
url_46 = 'https://www.quecochemecompro.com/precios/mercedes-eqb/'
genera_datos(url_46, 4)
url_47 = 'https://www.quecochemecompro.com/precios/mercedes-glb/'
genera_datos(url_47, 4)
url_48 = 'https://www.quecochemecompro.com/precios/mercedes-gle/'
genera_datos(url_48, 3)
url_49 = 'https://www.quecochemecompro.com/precios/mercedes-gle-hibrido/'
genera_datos(url_49, 3)
url_50 = 'https://www.quecochemecompro.com/precios/mercedes-gls/'
genera_datos(url_50, 3)
url_51 = 'https://www.quecochemecompro.com/precios/mitsubishi-outlander-hibrido/'
genera_datos(url_51, 2)
url_52 = 'https://www.quecochemecompro.com/precios/nissan-townstar/'
genera_datos(url_52, 3)
url_53 = 'https://www.quecochemecompro.com/precios/nissan-x-trail/'
genera_datos(url_53, 4)
url_54 = 'https://www.quecochemecompro.com/precios/nissan-x-trail-hibrido/'
genera_datos(url_53, 4)
url_55 = 'https://www.quecochemecompro.com/precios/opel-combo-life/'
genera_datos(url_55, 3)
url_56 = 'https://www.quecochemecompro.com/precios/opel-frontera/'
genera_datos(url_56, 2)
url_57 = 'https://www.quecochemecompro.com/precios/opel-frontera-electrico/'
genera_datos(url_57, 2)
url_58 = 'https://www.quecochemecompro.com/precios/opel-zafira-life-electrico/'
genera_datos(url_58, 2)
url_59 = 'https://www.quecochemecompro.com/precios/opel-zafira-tourer/'
genera_datos(url_59, 2)
url_60 = 'https://www.quecochemecompro.com/precios/peugeot-5008/'
genera_datos(url_60, 3)
url_61 = 'https://www.quecochemecompro.com/precios/peugeot-5008-hibrido/'
genera_datos(url_61, 3)
url_62 = 'https://www.quecochemecompro.com/precios/peugeot-5008-electrico/'
genera_datos(url_62, 3)
url_63 = 'https://www.quecochemecompro.com/precios/peugeot-rifter/'
genera_datos(url_63, 3)
url_64 = 'https://www.quecochemecompro.com/precios/peugeot-e-rifter-electrico/'
genera_datos(url_64, 3)
url_65 = 'https://www.quecochemecompro.com/precios/peugeot-traveller/'
genera_datos(url_65, 7)
url_66 = 'https://www.quecochemecompro.com/precios/range-rover/'
genera_datos(url_66, 5)
url_67 = 'https://www.quecochemecompro.com/precios/renault-espace/'
genera_datos (url_67, 3)
url_68 = 'https://www.quecochemecompro.com/precios/renault-grand-scenic/'
genera_datos(url_68, 3)
url_69 = 'https://www.quecochemecompro.com/precios/renault-kangoo-combi-electrico/'
genera_datos(url_60, 3)


200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK


In [14]:
url_70 = 'https://www.quecochemecompro.com/precios/seat-alhambra/'
genera_datos(url_70, 3)
url_71 = 'https://www.quecochemecompro.com/precios/seat-tarraco/'
genera_datos(url_71, 3)
url_72 = 'https://www.quecochemecompro.com/precios/skoda-enyaq/'
genera_datos(url_72, 3)
url_74 = 'https://www.quecochemecompro.com/precios/skoda-kodiaq/'
genera_datos(url_74, 4)
url_75 = 'https://www.quecochemecompro.com/precios/skoda-kodiaq-hibrido/'
genera_datos(url_75, 4)
url_76 = 'https://www.quecochemecompro.com/precios/swm-g03f/'
genera_datos(url_76, 2)
url_77 = 'https://www.quecochemecompro.com/precios/swm-g05f/'
genera_datos(url_77, 2)
url_78 = 'https://www.quecochemecompro.com/precios/toyota-highlander/'
genera_datos(url_78, 3)
url_79 = 'https://www.quecochemecompro.com/precios/toyota-proace-city-verso/'
genera_datos(url_79, 3)
url_80 = 'https://www.quecochemecompro.com/precios/toyota-proace-city-electrico/'
genera_datos(url_80, 3)
url_81 = 'https://www.quecochemecompro.com/precios/toyota-proace/'
genera_datos(url_81, 3)
url_82 = 'https://www.quecochemecompro.com/precios/volkswagen-caddy/'
genera_datos(url_82, 5)
url_83 = 'https://www.quecochemecompro.com/precios/volkswagen-california/'
genera_datos(url_83, 3)
url_84 = 'https://www.quecochemecompro.com/precios/volkswagen-caravelle/'
genera_datos(url_84, 3)
url_85 = 'https://www.quecochemecompro.com/precios/volkswagen-multivan/'
genera_datos(url_85, 5)
url_86 = 'https://www.quecochemecompro.com/precios/volkswagen-multivan-hibrido/'
genera_datos(url_86, 5)
url_87 = 'https://www.quecochemecompro.com/precios/volkswagen-tayron/'
genera_datos(url_87, 3)
url_88 = 'https://www.quecochemecompro.com/precios/volkswagen-tayron-hibrido/'
genera_datos(url_88, 3)
url_89 = 'https://www.quecochemecompro.com/precios/volkswagen-tiguan-allspace/'
genera_datos(url_89, 3)
url_90 = 'https://www.quecochemecompro.com/precios/volkswagen-touran/'
genera_datos(url_90, 3)
url_91 = 'https://www.quecochemecompro.com/precios/volvo-ex90/'
genera_datos(url_91, 4)
url_92 = 'https://www.quecochemecompro.com/precios/volvo-xc90/'
genera_datos(url_92, 3)
url_93 = 'https://www.quecochemecompro.com/precios/volvo-xc90-hibrido/'
genera_datos(url_93, 3)

200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK


In [15]:
# Añado algunos que aparecen en esta página y no los tenia recogidos
url_94 = 'https://www.quecochemecompro.com/precios/land-rover-defender/'
genera_datos(url_94, 3)
url_95 = 'https://www.quecochemecompro.com/precios/land-rover-defender-hibrido/'
genera_datos(url_95, 3)
url_96 = 'https://www.quecochemecompro.com/precios/mercedes-vito/'
genera_datos(url_96, 6)
url_97 = 'https://www.quecochemecompro.com/precios/renault-trafic/'
genera_datos(url_97, 3)
url_98 = 'https://www.quecochemecompro.com/precios/tesla-model-x/'
genera_datos(url_97, 2)
url_99 = 'https://www.quecochemecompro.com/precios/nissan-primastar/'
genera_datos(url_99, 2)
url_100 = 'https://www.quecochemecompro.com/precios/audi-sq7/'
genera_datos(url_100, 3)   
url_101 = 'https://www.quecochemecompro.com/precios/nissan-nv300/' 
genera_datos(url_101, 3)
url_102 = 'https://www.quecochemecompro.com/precios/byd-tang-electrico/'
genera_datos(url_102, 2)
url_103 = 'https://www.quecochemecompro.com/precios/land-rover-discovery/'
genera_datos(url_103, 2)
url_104 = 'https://www.quecochemecompro.com/precios/voyah-dream/'
genera_datos(url_104, 3)

200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK
200 OK


In [16]:
df = pd.DataFrame.from_dict(diccionario_coches)
df

,Marca,Modelo,Carroceria,Largo,Alto,Ancho,Peso,Maletero,Depósito,Potencia,...,Consumo,Combustible,Motor,Emisiones,Etiqueta,Precio,Valoración,Lo mejor,Lo mejorable,Opinión
0,Audi,[Q7],SUV XXL,5.072 m,1.743 m,1970 mm,2055 kg,867 l,85 l,231-507 CV,...,7.8-12.2 l/100Km,"[ Gasolina, MHEV Diesel, MHEV Gasolina]","[MHEV Diesel, AUDI Q7 3.0 55 TFSI MHEV QUATTRO...",205-272 g/Km,"[Etiqueta medioambiental C, Etiqueta medioambi...",74.460 €,"4,1","Espacio interior, calidad de construcción, tec...","[Proporciones exteriores algo raras, precio]","Uno de los SUVs más lujosos del mercado, puede..."
1,BMW,[X5],SUV XXL,4.935 m,1.765 m,2004 mm,2295 kg,650 l,80 l,298-625 CV,...,7.1-12.9 l/100Km,"[MHEV Diesel, MHEV Gasolina]","[MHEV Diesel, BMW X5 4.4 M COMPETITION MHEV, B...",189-291 g/Km,"[Etiqueta medioambiental C, Etiqueta medioambi...",94.037 €,"3,9","Tacto de conducción, tecnología, calidades, di...","[Precio elevado, opciones caras]","El BMW X5 es un SUV de lujo, de tamaño grande,..."
2,BMW,"[X5, Híbrido]",SUV XXL,4.935 m,1.755 m,2004 mm,2495 kg,500 l,69 l,489 CV,...,9.1 l/100Km,"[PHEV Gasolina, \n\n 24 de feb, 202...","[BMW X5 3.0 XDRIVE50E PHEV, PHEV Gasolina]",206 g/Km,"[Etiqueta medioambiental ECO, Etiqueta medioam...",103.041 €,"3,9","Imagen, prestaciones, autonomía en modo 100% e...","[Peso, precio de las opciones, tiempo de carga]",El BMW X5 cuenta con una versión híbrida enchu...
3,BMW,[X7],SUV XXL,5.151 m,1.835 m,2000 mm,2490 kg,750 l,83 l,352-530 CV,...,8.1-12.1 l/100Km,"[MHEV Diesel, MHEV Gasolina]","[MHEV Diesel, BMW X7 3.0 XDRIVE40I MHEV A, BMW...",213-274 g/Km,"[Etiqueta medioambiental C, Etiqueta medioambi...",120.510 €,"3,5",None,None,None
4,Citroën,"[C3, Aircross]",SUV Mediano,4.390 m,1.633 m,-,1298 kg,460 l,44 l,100-145 CV,...,5.3-6.2 l/100Km,"[ Gasolina, MHEV Gasolina]","[ Gasolina, CITROEN C3 AIRCROSS 1.2 MHEV PLUS,...",120-139 g/Km,"[Etiqueta medioambiental C, Etiqueta medioambi...",18.490 €,"2,9","Siete plazas, equipamiento disponible, precio ...","[Calidad de algunos plásticos, detalles de aca...",El nuevo C3 Aircross combina la estética de un...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,Audi,[SQ7],SUV XXL,5.702 m,1.743 m,1970 mm,2275 kg,887 l,85 l,507 CV,...,12.2 l/100Km,[ Gasolina],[ Gasolina],272 g/Km,"[Etiqueta medioambiental C, Etiqueta medioambi...",124.770 €,"3,8","Calidad y diseño interior, espacio, confort, t...","[Consumos altos en gasolina, autonomía]",El Audi SQ7 pertenece a una especie en peligro...
100,Nissan,[NV300],Comercial,4.999 m,1.970 m,1956 mm,1844 kg,-,80 l,110-170 CV,...,7.0-7.9 l/100Km,[ Diesel],[ Diesel],-,[Etiqueta medioambiental C],17.200 €,"3,0",None,None,None
101,BYD,"[Tang, eléctrico]",SUV XXL,4.875 m,1.745 m,1955 mm,2630 kg,235 l,-,517 CV,...,530 Kms,[ Eléctrico],[BYD TANG BEV 109KWH FLAGSHIP],0 g/Km,"[Etiqueta medioambiental eco, Etiqueta medioam...",72.000 €,"3,0","Calidad interior, confort de marcha, equipamie...",[Diseño algo anticuado frente a la competencia],El BYD Tang en un SUV 100% eléctrico que quier...
102,Land Rover,"[Rover, Discovery]",Todoterreno,4.970 m,1.888 m,2000 mm,2391 kg,258 l,89 l,249-350 CV,...,7.9-8.3 l/100Km,[MHEV Diesel],"[MHEV Diesel, LAND ROVER DISCOVERY 3.0 D350 MH...",207-219 g/Km,"[Etiqueta medioambiental C, Etiqueta medioambi...",83.370 €,"3,7","Capacidades todoterreno reales, confort, equip...",[Demasiado grande y pesado para ser ágil o div...,El Land Rover Discovery es un enorme SUV de lu...


In [17]:
# Voy a guardar el DataFrame en un csv para trabajar más cómodamente con él en otro notebook
df.to_csv('DF_caracteristicas.csv', index= False)